In [177]:
import pygplates
import numpy as np

In [178]:
#set path to rotation file, shapefiles, topologies etc.
loaddir = '/Users/Andrew/Documents/EarthByte_Models/Matthews++_410-0Ma'
savedir = '/Users/Andrew/Documents/PhD/Scripts/Scripts_Output/Test'
input_rotation_model = '%s/Global_EB_250-0Ma_GK07_Matthews_etal.rot' % loaddir
topology_features = ['%s/Global_EarthByte_Mesozoic-Cenozoic_plate_boundaries_Matthews_etal.gpml' % loaddir,
                     '%s/TopologyBuildingBlocks_AREPS.gpml' % loaddir]
coastlines_file = '%s/StaticGeometries/Coastlines/Global_coastlines_2015_v1_low_res.gpml' % loaddir

#load into pygplates
rotation_model = pygplates.RotationModel(input_rotation_model)
coastline_features = pygplates.FeatureCollection(coastlines_file)

In [181]:
start_time = 0
end_time = 10
timestep = 1
times = np.arange(start_time, end_time+timestep, timestep)

In [182]:
subduction_zone_geometries = dict()

for time in times:
    with open('%s/%s_geometry.csv' % (savedir, time), mode='w') as tmp_csv:
        csv_writer = csv.writer(tmp_csv, delimiter=',')
        #print tmp_csv
        subduction_zone_geometries[time] = []

        # Resolve our topological plate polygons (and deforming networks) to the current 'time'.
        # We generate both the resolved topology boundaries and the boundary sections between them.
        
        #these arrays are to store the topologies and boundary sections
        resolved_topologies = []
        shared_boundary_sections = []

        pygplates.resolve_topologies(topology_features, rotation_model, resolved_topologies, \
                                     time, shared_boundary_sections)

        # Iterate over the shared boundary sections (since one plate boundary can be shared by two plates)
        
        for  shared_boundary_section in shared_boundary_sections:
            
            #we only want subudction zones (NB you can change the feature type to transform/MoR etc.)
            if shared_boundary_section.get_feature().get_feature_type() == pygplates.FeatureType.gpml_subduction_zone:
                
                #iterate over sub segments in each feature
                for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
                    shared_sub_segment_geometry = shared_sub_segment.get_resolved_geometry()
                    
                    #convert to lat lon array, you can conver to a list or to a feature as well, depending
                    #what output you want one may be better than others
                    tmp = shared_sub_segment_geometry.to_lat_lon_array()
                
                #append everything to a big dictionary
                subduction_zone_geometries[time].append(tmp)

